In [1]:
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
from tensorflow.contrib.keras.python.keras.datasets import mnist
from tensorflow.contrib.keras.python.keras.utils import np_utils
from tensorflow.contrib.keras.python.keras.models import Sequential
from tensorflow.contrib.keras.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.contrib.keras.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.contrib.keras.python.keras.optimizers import SGD

In [15]:
def make_CNN_model(input_shape, output_size):
    # Initialization
    model = Sequential()
    
    # Convolution 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Convolution 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,  2)))
    model.add(Dropout(0.25))
    
    # Fully conected
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_size, activation='softmax'))
    
    return model

In [30]:
def preprocess(dataset):
    (x_train, y_train), (x_test, y_test) = dataset
    y_train_1hot = np_utils.to_categorical(y_train)
    y_test_1hot = np_utils.to_categorical(y_test)
    x_train_normed = (x_train / 255.).astype(np.float32)
    x_train_normed = np.expand_dims(x_train_normed, axis=-1)
    x_test_normed = (x_test / 255.).astype(np.float32)
    x_test_normed = np.expand_dims(x_test_normed, axis=-1)
    return (x_train_normed, y_train_1hot), (x_test_normed, y_test_1hot)

In [31]:
dataset = mnist.load_data()
(x_train, y_train), (x_test, y_test) = preprocess(dataset)

In [32]:
pixel_shape = x_train.shape[1:]
num_labels =  y_train.shape[-1]

In [33]:
x_train.shape

(60000, 28, 28, 1)

In [34]:
model = make_CNN_model(input_shape=pixel_shape, output_size=num_labels)

In [44]:
sgd = SGD(lr=0.01,
                  decay=1e-6,
                  momentum=0.9,
                  nesterov=True
                 )
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

In [38]:
model.fit(x_train,y_train,
                 batch_size=32,
                 epochs=10,
                 validation_data=(x_test, y_test)
                 )

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 248s - loss: 0.3761 - val_loss: 0.0521
Epoch 2/10
60000/60000 [==============================] - 280s - loss: 0.1271 - val_loss: 0.0350
Epoch 3/10
60000/60000 [==============================] - 289s - loss: 0.0976 - val_loss: 0.0314
Epoch 4/10
60000/60000 [==============================] - 286s - loss: 0.0837 - val_loss: 0.0333
Epoch 5/10
60000/60000 [==============================] - 301s - loss: 0.0753 - val_loss: 0.0222
Epoch 6/10
60000/60000 [==============================] - 283s - loss: 0.0685 - val_loss: 0.0218
Epoch 7/10
60000/60000 [==============================] - 289s - loss: 0.0602 - val_loss: 0.0227
Epoch 8/10
60000/60000 [==============================] - 289s - loss: 0.0587 - val_loss: 0.0216
Epoch 9/10
60000/60000 [==============================] - 300s - loss: 0.0572 - val_loss: 0.0191
Epoch 10/10
60000/60000 [==============================] - 302s - loss: 0.054

In [45]:
model.test_on_batch(x_test,y_test)

[0.018248521, 0.99449998]

In [46]:
model.save('cnn_model.h5')